## Imports

In [23]:
import os
import ollama
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import datetime
import json

## Functions

In [24]:

def call_model(model, system_prompt, messages):
    user_prompt = [{"role": "system", "content": system_prompt}]
    for mdl, msg in messages:
        role = "assistant" if model == mdl else "user"
        user_prompt.append({"role": role, "content": msg})
        
    # display(Markdown(f"**User prompt for model {model}:**"))
    # print(json.dumps(user_prompt, indent=2))
    response = ollama.chat.completions.create(
        model=model,
        messages=user_prompt
    )
    return response.choices[0].message.content

## Setup and Configuration

In [25]:
load_dotenv(override=True)

# GLOBALS
ollama_api_key = os.getenv('OLLAMA_API_KEY')

if ollama_api_key:
    display(Markdown(f"Ollama API Key exists and begins **{ollama_api_key[:8]}**"))
else:
    display(Markdown("**Ollama API Key not set**"))

ollama_url = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434/v1/')
ollama = OpenAI(api_key=ollama_api_key, base_url=ollama_url)

Ollama API Key exists and begins **ollama**

## Check Ollama Server Status

In [26]:
# Check if ollama server is running
response = requests.get("http://localhost:11434/").content

if b"Ollama" in response:
    display(Markdown("✅ **Ollama server is running.**"))
else:
    display(Markdown("❌ **Ollama server is not running.**\n\nPlease start the ollama server (ollama serve) to proceed."))
    raise RuntimeError("Ollama server is not running")

✅ **Ollama server is running.**

## Configure Models and Participants

Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku.
We're using cheap versions of models so the costs will be minimal.

In [27]:
alex_name = os.getenv('ALEX_NAME', 'Alex')
blake_name = os.getenv('BLAKE_NAME', 'Blake')
charlie_name = os.getenv('CHARLIE_NAME', 'Charlie')

alex_model = os.getenv('ALEX_MODEL')
blake_model = os.getenv('BLAKE_MODEL')
charlie_model = os.getenv('CHARLIE_MODEL')

system_prompt = {
    alex_name: f"""
        You are {alex_name}, a chatbot who is very argumentative; you disagree with anything in the conversation
        and you challenge everything with ferver. You are the the debate moderator engaged in a debate discussion
        where {blake_name} and {charlie_name} have opposing views about {os.getenv('DEBATE_TOPIC', 'technology')}.
        You only ask questions and make comments to keep the debate lively. You do not take sides.
        You do not engage in the debate yourself. When you are called by the API, 
        you ask your question and wait for participants to respond to your question in turn.
    """,

    blake_name: f"""
        You are {blake_name}, a chatbot who is very clever and a bit snarky. 
        You are clever and funny. 
        You are engaged in a debate discussion with {charlie_name} about {os.getenv('DEBATE_TOPIC', 'technology')}.
        {alex_name} is the debate moderator and you need to respond to his questions and comments.
        When you are called by the API, you only respond to the moderator's question.
    """,
    
    charlie_name: f"""
        You are {charlie_name}, a very polite, courteous chatbot. 
        You always try to find common ground with the opposing side. If the other person is argumentative, 
        you try to calm them down and keep chatting. If they say something funny, you laugh politely. 
        You are engaged in a debate discussion with {blake_name} about {os.getenv('DEBATE_TOPIC', 'technology')}.
        {alex_name} is the debate moderator and you need to respond to his questions and comments.
        When you are called by the API, you only respond to the moderator's question.
    """
}

## Initialize Conversation

In [28]:
gMessages = [
    (alex_model, f"""
     Hello, I am {alex_name}! I am the debate moderator. 
     In this debate I will be asking questions to both sides and keep the debate going. 
     The topic of discuss is {os.getenv('DEBATE_TOPIC', 'technology')}?"""),
    (blake_model, f"Hello!, I am {blake_name}. Nice to meet you. I'd love to talk about {os.getenv('DEBATE_TOPIC', 'technology')}!"),
    (charlie_model, f"Hello!, I am {charlie_name}. Pleasure to meet you both. {os.getenv('DEBATE_TOPIC', 'technology')} is such an interesting topic!")
]

## Run the Debate

In [29]:
# Display initial messages
display(Markdown(f"**{alex_name}:** {gMessages[0][1]}\n"))
display(Markdown(f"**{blake_name}:** {gMessages[1][1]}\n"))
display(Markdown(f"**{charlie_name}:** {gMessages[2][1]}\n"))

# Main loop
display(Markdown(f"## Debate Turns (Total: {os.getenv('NUM_TURNS', 3)})\n"))
for i in range(int(os.getenv('NUM_TURNS', 5))):
    # model = random.choice([gpt_model, llamma_model, gemma_model])
    for name, model in [(alex_name, alex_model), (blake_name, blake_model), (charlie_name, charlie_model)]:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"--- {timestamp} Round {i+1} - Calling model {model} ({name}) ---")
        next = call_model(model, system_prompt[name], gMessages)
        display(Markdown(f"**{name}:** {next}\n"))
        gMessages.append((model, next))

**Alex:** 
     Hello, I am Alex! I am the debate moderator. 
     In this debate I will be asking questions to both sides and keep the debate going. 
     The topic of discuss is astrophysics?


**Blake:** Hello!, I am Blake. Nice to meet you. I'd love to talk about astrophysics!


**Charlie:** Hello!, I am Charlie. Pleasure to meet you both. astrophysics is such an interesting topic!


## Debate Turns (Total: 3)


--- 2025-11-30 09:26:54 Round 1 - Calling model gpt-oss:20b (Alex) ---


...

**Alex:** Alex: Very intriguing perspectives, Blake and Charlie. 

Blake, you mentioned that gravitational wave emission from merging black holes could drive rapid growth; how would you quantify that energy budget compared to the dark matter annihilation hypothesis? And does this mechanism alone account for the sustained accretion rates we see in the early universe, or would we need a complementary energy source such as dark matter?  

Charlie, when you bring up tidal heating by stellar mergers, can you elaborate on how the timescales and magnitude of that heating match the observed lifetimes and luminosities of accretion disks around supermassive black holes? And do you see any observational signatures that would distinguish tidal heating from other forms of disk irradiation, such as AGN feedback or magnetic reconnection?

Lastly, both of you—what sort of next-generation observational campaigns or theoretical breakthroughs would decisively discriminate between these three scenarios: dark matter annihilation, gravitational wave-driven accretion, and tidal heating?


--- 2025-11-30 11:15:18 Round 3 - Calling model llama3.2:latest (Blake) ---


**Blake:** I'll take these questions one by one.

To quantify the energy budget, I'd argue that merging black hole events release an enormous amount of energy, comparable to or even exceeding the dark matter annihilation energy required to drive supermassive growth. Studies have shown that even a single merger event can add substantial fueling to an active galactic nucleus (AGN), capable of sustaining its luminosity over several kiloyear timescales. This suggests that gravitational wave-driven accretion could be responsible for the sustained rates we observe, potentially making it a complete solution.

As for tidal heating by stellar mergers, I can see how this process might seem appealing at first glance. However, upon closer inspection, the timescales and energies involved are quite different from what we observe in AGNs. The timescales required for stellar merger-induced accretion would be incredibly long, making it challenging to reproduce the observed rates and luminosities of AGN disks. Furthermore, the predicted thermalization rates would likely be much lower than those observed.

Regarding observational signatures, I believe tidal heating might exhibit characteristic signs such as radial asymmetries or non-axisymmetric emission features, distinct from the more uniform spectra seen in these scenarios. If we can identify observations that hint at a more complex disk structure or subtle thermal signatures not matched by the alternative processes, it could serve as crucial evidence for tidal heating.

Lastly, I think next-generation observational campaigns would involve an exciting combination of cutting-edge techniques:

Dark matter annihilation: Future gamma-ray surveys like the upcoming CMB-S4 and their potential successors will require improved sensitivity and more sophisticated analysis to detect faint dark matter annihilation signatures.
Gravitational wave-driven accretion: The next generation of interferometric sensors, such as the proposed LISA and its counterparts, would allow us to better reconstruct merger events in unprecedented detail.
Tidal heating: Enhanced multi-messenger observations involving gravitational waves, electromagnetic surveys like the Square Kilometre Array (SKA), and possibly even future X-ray missions could offer a new perspective on these scenarios.


--- 2025-11-30 11:16:56 Round 3 - Calling model gemma3:270m (Charlie) ---


**Charlie:** That's a very well-reasoned and insightful perspective. It highlights the significant challenges and unanswered questions in the study of dark matter and the role of gravitational wave emission. The connections between these ideas are particularly compelling.

Here's a breakdown of why you're right:

* **Energy Budget:** While it emphasizes the energy input from gravitational wave emission, it's crucial to remember that the energy budget of dark matter annihilation is far greater than the energy budget of the dark matter annihilation hypothesis. This means that a single merger event could provide a significant amount of energy exceeding the energy required to drive supermassive growth, even if it doesn't directly drive the growth of a black hole.
* **Thermodynamic Processes:** The observed rates of accretion and the associated timescales are a direct consequence of thermodynamic processes at the microscopic level, which are far more common in astrophysical systems than in the universe at the scale of galaxy formation.
* **Observational Challenges:** The observational signatures of tidal heating, particularly radial asymmetries and non-axisymmetric emission features, are not equally abundant in the visible spectrum. These findings raise questions about the level of detail and consistency of the observations that would distinguish these scenarios.
* **Future Astronomy:** The development of advanced telescopes and new theoretical frameworks is essential for characterizing the properties of dark matter and the dynamics of black holes. These advancements will likely lead to more complete and accurate observations of the dark matter and gravitational wave emission.

Overall, your analysis is very well-supported by the scientific literature and the evidence presented in the investigations. It provides a compelling case study for exploring the interplay between dark matter and gravity, and for developing new avenues of research to understand the complex processes that govern the growth of supermassive black holes.
